In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def double2im(mat: np.ndarray):
    return (mat * 255).astype(np.uint8)

def im2double(img: np.ndarray):
    return img / 255.

def plot_img(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.axis('off')
    plt.show()

In [ ]:
import numpy as np 
import cv2
code=[
    np.array([[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]),
    np.array([[0,0,0,0],[0,255,0,0],[0,0,0,0],[0,0,0,0]]),
    np.array([[0,0,0,0],[0,255,255,0],[0,0,0,0],[0,0,0,0]]),
    np.array([[0,0,0,0],[0,255,255,0],[0,255,0,0],[0,0,0,0]]),
    np.array([[0,0,0,0],[0,255,255,0],[0,255,255,0],[0,0,0,0]]),
    np.array([[0,255,0,0],[0,255,255,0],[0,255,255,0],[0,0,0,0]]),
    np.array([[0,255,0,0],[0,255,255,255],[0,255,255,0],[0,0,0,0]]),
    np.array([[0,255,0,0],[0,255,255,0],[0,255,255,0],[0,0,255,0]]),
    np.array([[0,255,0,0],[0,255,255,255],[255,255,255,0],[0,0,255,0]]),
    np.array([[0,255,255,0],[0,255,255,255],[255,255,255,0],[0,0,255,0]]),
    np.array([[0,255,255,0],[0,255,255,255],[255,255,255,255],[0,0,255,0]]),
    np.array([[0,255,255,0],[0,255,255,255],[255,255,255,255],[0,255,255,0]]),
    np.array([[0,255,255,0],[255,255,255,255],[255,255,255,255],[0,255,255,0]]),
    np.array([[0,255,255,255],[255,255,255,255],[255,255,255,255],[0,255,255,0]]),
    np.array([[0,255,255,255],[255,255,255,255],[255,255,255,255],[0,255,255,255]]),
    np.array([[0,255,255,255],[255,255,255,255],[255,255,255,255],[255,255,255,255]]),
    np.array([[255,255,255,255],[255,255,255,255],[255,255,255,255],[255,255,255,255]]),
]

In [ ]:
jarvis=[
       (0, 1, 7 / 48),
        (0, 2, 5 / 48),
        (1, -2, 3 / 48),
        (1, -1, 5 / 48),
        (1, 0, 7 / 48),
        (1, 1, 5 / 48),
        (1, 2, 3 / 48),
        (2, -2, 1 / 48),
        (2, -1, 3 / 48),
        (2, 0, 5 / 48),
        (2, 1, 3 / 48),
        (2, 2, 1 / 48),
]
Stucki =  [
        (1, 0, 8 / 42),
        (2, 0, 4 / 42),
        (-2, 1, 2 / 42),
        (-1, 1, 4 / 42),
        (0, 1, 8 / 42),
        (1, 1, 4 / 42),
        (2, 1, 2 / 42),
        (-2, 2, 1 / 42),
        (-1, 2, 2 / 42),
        (0, 2, 4 / 42),
        (1, 2, 2 / 42),
        (2, 2, 1 / 42),
]

In [ ]:
def error_diffusion(image: np.ndarray,
                    filtre: list[tuple[int, int, float]],
                    kernel: int,
                    thres: int = 127):
    height, width = image.shape[:2]
    padded_shape = (height + 2 * kernel, width + 2 * kernel)
    padded = np.full(padded_shape, thres)
    padded[kernel:-kernel, kernel:-kernel] = image
    rst = padded.copy()

    for i in range(kernel, padded.shape[0]-kernel):
        for j in range(kernel, padded.shape[1]-kernel):
            rst[i, j] = 255 if padded[i, j] > thres else 0
            # if padded[i, j] > thres:
            #     rst[i, j] = 255
            # else:
            #     rst[i, j] = 0
            error = int(-rst[i][j] + padded[i][j])
            for (dx, dy, val) in filtre:
                padded[i+dx][j+dy] += int(error * val)

    return rst[kernel:-kernel, kernel:-kernel]

In [ ]:
from pathlib import Path

original = cv2.imread(Path('data/test', 'input_image.png'))


for i in range(15):
    filename = f'FinalLayers_{i:02d}.png'
    rgb_image = cv2.imread(Path('data/test', filename), cv2.IMREAD_UNCHANGED)
    alpha = rgb_image[..., -1] / 255.
    height, width = rgb_image.shape[:2]
    image2 = np.zeros(shape=(height, width, 3))

    for c in range(3):
        # image = cv2.cvtColor(rgb_image[..., c], cv2.COLOR_BGR2GRAY) * alpha
        image = rgb_image[..., c] * alpha
        image2[..., c] = error_diffusion(image, Stucki, 4, thres=64)

    if not i:
        output = np.zeros((height, width, 3))

    output += image2

    cv2.imwrite(Path('data/results', filename), image2)

    # output[:, :, 0] += rgb_image[..., 0] * alpha
    # output[:, :, 1] += rgb_image[..., 1] * alpha
    # output[:, :, 2] += rgb_image[..., 2] * alpha

    # plt.figure(figsize=(10, 10))
    plt.figure(figsize=(15, 7))
    plt.subplot(1, 3, 1)
    plt.imshow(rgb_image)
    # plt.imshow(image, cmap="gray")
    plt.axis('off')
    plt.subplot(1, 3, 2)
    plt.imshow(image2)
    plt.axis('off')
    # plt.subplot(1, 3, 3)
    # plt.imshow(image3)
    # plt.axis('off')

    plt.suptitle(filename)
    plt.show()

    # break

# output = output.astype(np.uint8)
# output[output > 255] = 255
# plt.imshow(output)
# plt.show()

In [ ]:
from pathlib import Path

original = cv2.imread(Path('data/test', 'input_image.png'))


for i in range(15):
    filename = f'FinalLayers_{i:02d}.png'
    rgb_image = cv2.imread(Path('data/test', filename), cv2.IMREAD_UNCHANGED)
    alpha = rgb_image[..., -1] / 255.
    for c in range(3):
        image = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2GRAY) * alpha

        image2 = error_diffusion(image, Stucki, 4, thres=64)
        height, width = image2.shape

        if not i:
            output = np.zeros((height, width))

        output += image2

    # output[:, :, 0] += rgb_image[..., 0] * alpha
    # output[:, :, 1] += rgb_image[..., 1] * alpha
    # output[:, :, 2] += rgb_image[..., 2] * alpha

    # plt.figure(figsize=(10, 10))
    plt.figure(figsize=(15, 7))
    plt.subplot(1, 3, 1)
    plt.imshow(rgb_image)
    # plt.imshow(image, cmap="gray")
    plt.axis('off')
    plt.subplot(1, 3, 2)
    plt.imshow(image2)
    plt.axis('off')
    # plt.subplot(1, 3, 3)
    # plt.imshow(image3)
    # plt.axis('off')

    plt.suptitle(filename)
    plt.show()

    # break

# output = output.astype(np.uint8)
# output[output > 255] = 255
# plt.imshow(output)
# plt.show()

In [ ]:
# output = output.astype(np.uint8)
# output[output > 255] = 255

cv2.imwrite(Path('data/results', 'output.png'), output)

# plt.figure(figsize=(10, 15))
# plt.imshow(output, cmap='gray')
# plt.show()

In [ ]:
filename = f'FinalLayers_01.png'
rgb_image = cv2.imread(Path('data/test', filename), cv2.IMREAD_UNCHANGED)
alpha = rgb_image[..., -1] / 255.
image = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2GRAY) * alpha
# image = cv2.imread(Path('data/test', filename), 0)

image2 = error_diffusion(image, jarvis, 4)

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(image, cmap="gray")
# plt.subplot(1, 3, 2)
# plt.imshow(image1, cmap="gray")
plt.subplot(1, 3, 3)
plt.imshow(image2, cmap="gray")

# plt.suptitle(filename)
plt.show()

In [ ]:
filename = f'FinalLayers_02.png'
rgb_image = cv2.imread(Path('data/test', filename))


plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(rgb_image[:64, :64, 0])
plt.subplot(1, 3, 2)
plt.imshow(rgb_image[:64, :64, 1])
plt.subplot(1, 3, 3)
plt.imshow(rgb_image[:64, :64, 2])
# plt.colorbar()  # Add a colorbar

In [ ]:
from tqdm import tqdm


for i in tqdm(range(15)):
    filename = f'FinalLayers_{i:02d}.png'
    image = cv2.imread(Path('data/test', filename), cv2.IMREAD_UNCHANGED)
    alpha = image[..., -1] / 255.
    image = cv2.cvtColor()

    if not i:
        output = np.zeros(shape=image.shape)

    image1 = tramme_ordonne(image, code, 4)
    image2 = error_diffusion(image, Stucki, 4)

    output += image2

    # plt.figure(figsize=(10, 10))
    # plt.figure(figsize=(15, 5))
    # plt.subplot(1, 3, 1)
    # plt.imshow(image, cmap="gray")
    # plt.subplot(1, 3, 2)
    # plt.imshow(image1, cmap="gray")
    # plt.subplot(1, 3, 3)
    # plt.imshow(image2, cmap="gray")

    # plt.suptitle(filename)
    # plt.show()

output = output.astype(np.uint8)
output[output > 255] = 255
plt.imshow(output[:128,:128], cmap='gray')
plt.show()

In [ ]:

plt.imshow(output[:128,:128], cmap='gray')
plt.show()

In [ ]:
filename = f'FinalLayers_12.png'
image = cv2.imread(Path('data/test', filename), cv2.IMREAD_UNCHANGED)
alpha = image[..., -1]
plt.imshow(alpha)

# image1 = tramme_ordonne(image, code, 4)
# image2 = error_diffusion(image, Stucki, 4)

# plt.figure(figsize=(10, 10))
# plt.figure(figsize=(15, 5))
# plt.subplot(1, 3, 1)
# plt.imshow(image[128:256, 128:256], cmap="gray")
# plt.subplot(1, 3, 2)
# plt.imshow(image1[512:1024, 512:1024], cmap="gray")
# plt.subplot(1, 3, 3)
# plt.imshow(image2[128:256, 128:256], cmap="gray")

# plt.suptitle(filename)
# plt.show()

In [ ]:
import numpy as np
import cv2
from pathlib import Path

In [ ]:
def add_paper_texture(source: np.ndarray, texture: np.ndarray):
    source = np.array(source)
    texture = np.array(texture)
    texture = cv2.resize(texture, (source.shape[1], source.shape[0]))

    norm_source = source / 255.0
    norm_texture = texture / 255.0

    norm_target = norm_source * norm_texture
    target = (norm_target * 255).astype(np.uint8)

    return target


In [ ]:
texture_path = Path('data', 'papers')
source_path = Path('data',  'test', 'recolor_drac_new.png')

for i, texture_path in enumerate(texture_path.iterdir()):
    source = cv2.imread(source_path)
    texture = cv2.imread(texture_path)
    
    target = add_paper_texture(source, texture)

    print(texture_path)
    cv2.imwrite(Path('output', f'blended_image_{i}.jpg'), target)